Giải capcha

In [53]:
from capmonstercloudclient import CapMonsterClient, ClientOptions
from capmonstercloudclient.requests import RecaptchaV2ProxylessRequest

API_KEY = "b238f538e55b7deb0da93267f61d8763"
WEBSITE_URL = 'https://af.uppromote.com/solar-power-store-canada/login'
WEBSITE_KEY = '6LcfFqkaAAAAAODkHHT2DLE7UBeSbf7kVCdBkTQE'
async def solve_captcha():
    client_options = ClientOptions(api_key=API_KEY)
    cap_monster_client = CapMonsterClient(options=client_options)
    recaptcha2request = RecaptchaV2ProxylessRequest(
        websiteUrl=WEBSITE_URL,
        websiteKey=WEBSITE_KEY
    )
    return await cap_monster_client.solve_captcha(recaptcha2request)


Login


In [54]:
import re
import aiohttp
import re

def extract_user_id(html_content):
    html_str = html_content.decode()
    match = re.search(r'id=([a-f\d]+)', html_str)
    if match:
        return match.group(1)
    else:
        return None

def extract_cookies_from_header(set_cookie_header):
    # Extract cookies from Set-Cookie headers
    cookies = re.findall(r'Set-Cookie: (.*?);', set_cookie_header)

    # Join the cookies into a single string
    cookies_string = "; ".join(cookies)

    return cookies_string

async def LoginAndGetTokenAsync(login_url, payload, headers): 
    async with aiohttp.ClientSession() as session:
        async with session.post(login_url, data=payload, headers=headers, allow_redirects=False) as response:
            if response.status == 302:
                cookies = response.cookies
                if cookies:
                    return extract_cookies_from_header(str(cookies))
                else:
                    print("Không tìm thấy token.")
                    return None, None
            else:
                print(f"Error {response.status}: {response.text()}")
                response.raise_for_status()
                return None, None


In [55]:
url = 'https://af.uppromote.com/solar-power-store-canada/login_aff'

res = await solve_captcha()
payload = {
    "_token": "5xhC3w0BB9Cezh7fhtZJg5YaHJvyInW5AC4qy8Mi",
    "shop_id": "80375",
    "email": "teamasmads@gmail.com",
    "password": "2N*G5k$7ux5j2!F",
    "g-recaptcha-response": res["gRecaptchaResponse"]
}
headers = {}

Cookie = await LoginAndGetTokenAsync(url, payload, headers)
print(f"Cookie= {Cookie}")


Cookie= XSRF-TOKEN=eyJpdiI6IjBrTTk3TjhqcmpUU09VQ3J6NEN6NUE9PSIsInZhbHVlIjoidFFQZ3JRSnhsSFdkS1hiSEpBZjM2SmxjbW5tdzlFXC9hWElYNURaRnA1eU8rWUw2ejNCdHBQVDYrRExCTVk2SXUiLCJtYWMiOiJlY2E3MWY2ZmZjMjAwYzkzZGU4NjRmOWRmNzEyNWM3ODYxNjI3YzQ4YTk1NmE3ZjRkODBmNjhkMGJiNjYzODcxIn0%3D; laravel_session=eyJpdiI6ImhoaWl2bWNHUE9tdEhZdzFGbzVqVUE9PSIsInZhbHVlIjoiUnVpSlMzQ2tZSzU4ZVU0RnNxXC9wdUYzQ2t0SGV0cVVhSzdcL1V5Rm53aEorM25cL0hNU0NvWWplUU9IWHQxdmFzMmxqd2tQRzFMamdXcVZNWTk5XC9YXC96VTlhVTBKdmdcL1BVdnNBVGJmZGdkXC82aXF6ZFdNdFJGNStyMlVcL3I2UnF1QSIsIm1hYyI6IjE3YWIzYzUyNDQ5MTVjYjU0NjlhZDY3NzE5NzI4M2FhMzRlY2EyMzQ0MzE5MjE4ZGZlMWRiNzc1NjYzMTdmZWUifQ%3D%3D


GET data


In [56]:
import aiohttp
from bs4 import BeautifulSoup

async def fetch_data(url, cookie):
    headers = {'Cookie': cookie}
    async with aiohttp.ClientSession() as session:
        async with session.get(url, headers=headers) as response:         
            if response.status == 200:
                html_content = await response.text()
                soup = BeautifulSoup(html_content, 'html.parser')
                
                selected_elements = soup.select('#commission .panel-body__pending, #commission .panel-body__approved, #commission .panel-body__paid')
                return {label_element.text.strip(): element.text.strip() for element in selected_elements if (label_element := element.find_next(class_='my-0')) is not None}
            else:
                print(f"Error: {response.status}")
                return None
            
await fetch_data('https://af.uppromote.com/solar-power-store-canada/dashboard', Cookie)


{'Pending': '$0', 'Approved': '$0', 'Paid': '$0'}

Crawl data func

In [57]:

async def crawl_data(args):
    url, email, password = args

    res = await solve_captcha()
    payload = {
        "_token": "5xhC3w0BB9Cezh7fhtZJg5YaHJvyInW5AC4qy8Mi",
        "shop_id": "80375",
        "email": email,
        "password": password,
        "g-recaptcha-response": res["gRecaptchaResponse"]
    }
    headers = {}

    cookie = await LoginAndGetTokenAsync('https://af.uppromote.com/solar-power-store-canada/login_aff', payload, headers)

    return await fetch_data('https://af.uppromote.com/solar-power-store-canada/dashboard', cookie)


Main

In [58]:
import pypeln as pl

data = [("https://af.uppromote.com/solar-power-store-canada/login", "teamasmads@gmail.com", "2N*G5k$7ux5j2!F")]
stage = await pl.task.map(crawl_data, data, workers=100)
print(stage)


[{'Pending': '$0', 'Approved': '$0', 'Paid': '$0'}]
